# NLP- Code for Word2Vec neural network(Tensorflow)
## by Madhu Sanjeevi

https://medium.com/deep-math-machine-learning-ai/chapter-9-2-nlp-code-for-word2vec-neural-network-tensorflow-544db99f5334

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/learning-stack/Colab-ML-Playbook/blob/master/NLP/NLP2%20-%20Code%20for%20Word2Vec/Word2Vec.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/learning-stack/Colab-ML-Playbook/blob/master/NLP/NLP2%20-%20Code%20for%20Word2Vec/Word2Vec.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import codecs

!wget https://raw.githubusercontent.com/mneedham/neo4j-himym/master/data/import/sentences.csv

input_file = codecs.open("sentences.csv", "r",encoding='utf-8', errors='replace')

data = pd.read_csv(input_file)

In [0]:
data.head()

In [0]:
df= data.loc[data['Season']==1]
df=df.fillna("(")

In [0]:
df['Character']=""
for sen,id in zip(df["Sentence"],df["SentenceId"]):
    
    if "[" is sen[0] or "(" is sen[0] or "Scene" in sen[0:5] or ":" not in sen:
        df = df.drop(df[df['Sentence']==sen].index)
        
    else:
        value=""
        for c in sen:
            if c==":":
                df.set_value(id-1, 'Sentence', sen[len(value)+1:])
                break
                
            else:
                value+=c
                
        
        if "(" in value:
            index=value.index("(")
            value=value[:index-1]
            
        value=value.lower()    
        #df.loc[df['Sentence'] == sen,'Character']=sen #setting the values
        df.set_value(id-1, 'Character', value)

#re arrange the order 
df = df[['SentenceId','EpisodeId','Season','Episode','Character','Sentence']]

In [0]:
df[:5]

In [0]:
# for labeling in plotting
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.02*height,
                '%d' % int(height),
                ha='center', va='bottom')

topchar=df.Character.value_counts()[:7]
labels=np.array(topchar.keys())
values=np.array(topchar)
ind=np.arange(len(labels))

fig,ax=plt.subplots()
rects=ax.bar(ind,values,color='g')
ax.set_xticklabels(labels,rotation='vertical')
ax.set_xticks(ind)
ax.set_ylabel("Count")
ax.set_title("Top characters with most dialogs")
autolabel(rects)
plt.show()

In [0]:
#pre processing the text 
import re
def normalize_text(text):

    text=text.lower()
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(pic\.twitter\.com/[^\s]+))','', text)
    text = re.sub('@[^\s]+','', text)
    text = re.sub('#([^\s]+)', '', text)
    text = re.sub('[:;>?<=*+()&,\-#!$%\{˜|\}\[^_\\@\]1234567890’‘]',' ', text)
    text = re.sub('[\d]','', text)
    text = text.replace(".", '')
    text = text.replace("'", '')
    text = text.replace("`", '')
    text = text.replace("'s", '')
    text = text.replace("/", ' ')
    text = text.replace("\"", ' ')
    text = text.replace("\\", '')
    re.sub(' +', ' ', text)
    text=text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    #normalize some utf8 encoding
    text = text.replace("\x9d",'').replace("\x8c",'')
    text = text.replace("\xa0",'')
    text = text.replace("\x9d\x92", '').replace("\x9a\xaa\xf0\x9f\x94\xb5", '').replace("\xf0\x9f\x91\x8d\x87\xba\xf0\x9f\x87\xb8", '').replace("\x9f",'').replace("\x91\x8d",'')
    text = text.replace("\xf0\x9f\x87\xba\xf0\x9f\x87\xb8",'').replace("\xf0",'').replace('\xf0x9f','').replace("\x9f\x91\x8d",'').replace("\x87\xba\x87\xb8",'')
    text = text.replace("\xe2\x80\x94",'').replace("\x9d\xa4",'').replace("\x96\x91",'').replace("\xe1\x91\xac\xc9\x8c\xce\x90\xc8\xbb\xef\xbb\x89\xd4\xbc\xef\xbb\x89\xc5\xa0\xc5\xa0\xc2\xb8",'')
    text = text.replace("\xe2\x80\x99s", "").replace("\xe2\x80\x98", '').replace("\xe2\x80\x99", '').replace("\xe2\x80\x9c", "").replace("\xe2\x80\x9d", "")
    text = text.replace("\xe2\x82\xac", "").replace("\xc2\xa3", "").replace("\xc2\xa0", "").replace("\xc2\xab", "").replace("\xf0\x9f\x94\xb4", "").replace("\xf0\x9f\x87\xba\xf0\x9f\x87\xb8\xf0\x9f", "")
    text =  re.sub(r"\b[a-z]\b", "", text)
    text=re.sub( '\s+', ' ', text).strip()
    
    text=re.sub(r'\.+', ".", text)
    text=re.sub(r'\.\.+', ' ', text).replace('.', '')
    # Replace multiple dots with space
    text = re.sub('\.\.+', ' ', text) 
    # Remove single dots
    text = re.sub('\.', '', text)
    text = re.sub(r'\.{2,}', ' ', text)
    text = re.sub(r'\.{1}', '', text)
    
    return text

So the sentence column is the actual raw data , we need to normalize the data ( removing symbols, spaces and etc…)

In [0]:
sentences=df["Sentence"].tolist()
normalized_sentences=[]
for sentence in sentences:
    norm_sent=normalize_text(sentence)
    normalized_sentences.append(norm_sent)

so now we have the clean data with us , let’s create a dictionary out of these sentences ( here I take unigrams (1 word) ) you can take bi-grams also,

In [0]:
import collections
# sentences to words and count
words=" ".join(normalized_sentences).split() 
count= collections.Counter(words).most_common() 
print ("Word count", count[:5])

In [0]:
# Build dictionaries
unique_words =  [i[0] for i in count]
dic = {w: i for i, w in enumerate(unique_words)} #dic, word -> id cats:0 dogs:1 ......
voc_size = len(dic)

In [0]:
voc_size

here we first identified the unique words by taking the count then we created the dictionary. ( word order depends based on the count).

let me take the first sentence in our data set to see how it looks

In [0]:
# Make indexed word data
data = [dic[word] for word in words] #count rank for every word in words
print('Sample data', data[:10], words[:10])

We just replaced the words with numbers , remember these numbers are not the vectors , these are just the indexes in our dictionary.

Now let’s create the word embeddings ( word2 vec)

To build the model we can use skip gram ( as we discussed in the last story)

Continuous Bag-of-Words model (CBOW)
It predicts one word based on the surrounding words (it takes an entire context as an observation or takes a window sized context as an observation)

Ex: Text= “Mady goes crazy about machine leaning” and window size is 3

it takes 3 words at a time predicts the center word based on the surrounding words → [ [“Mady”,”crazy” ] , “goes”] → “goes” is the target word , and the other two are inputs.

Skip-Gram model
It takes one word as input and try to predict the surrounding (neighboring) words,

[“Mady”, “goes”],[“goes”,”crazy”] → “goes” is the input word and “Mady” and “Crazy” are the surrounding words (Output probabilities)

In [0]:
# Let's make a training data for window size 1 for simplicity

cbow_pairs = []
for i in range(1, len(data)-1) :
    cbow_pairs.append([[data[i-1], data[i+1]], data[i]]);
    
print('Context pairs rank ids', cbow_pairs[:5])
print()

cbow_pairs_words = []
for i in range(1, len(words)-1) :
    cbow_pairs_words.append([[words[i-1], words[i+1]], words[i]]);
print('Context pairs words', cbow_pairs_words[:5])

now we have the skip gram pairs as X and y values , lets create a functions that gives us the pairs batch wise

In [0]:
# Let's make skip-gram pairs
# (quick, the), (quick, brown), (brown, quick), (brown, fox), ...
skip_gram_pairs=[]

for c in cbow_pairs:
    skip_gram_pairs.append([c[1],c[0][0]])
    skip_gram_pairs.append([c[1],c[0][1]])
print('skip-gram pairs', skip_gram_pairs[:5])
print()
skip_gram_pairs_words=[]
for c in cbow_pairs_words:
    skip_gram_pairs_words.append([c[1],c[0][0]])
    skip_gram_pairs_words.append([c[1],c[0][1]])
print('skip-gram pairs words', skip_gram_pairs_words[:5])

In [0]:
def get_batch(size):
    assert size<len(skip_gram_pairs)
    X=[]
    Y=[]
    rdm = np.random.choice(range(len(skip_gram_pairs)),size,replace=False)
    
    for r in rdm:
        X.append(skip_gram_pairs[r][0])
        Y.append([skip_gram_pairs[r][1]])
    return X , Y

# generate_batch test
print ('Batches (x, y)', get_batch(3))

Now we have the batch inputs to feed to Neural network so let’s build the neural network using tensorflow

In [0]:
#tensor flow code
import tensorflow as tf

batch_size = 20
embedding_size = 2
num_sampled = 15    # Number of negative examples to sample.

X= tf.placeholder(tf.int32,shape=[batch_size]) #inputs
Y= tf.placeholder(tf.int32,shape=[batch_size,1]) #labels

with tf.device("/cpu:0"):
      # Look up embeddings for inputs.
    embeddings = tf.Variable(tf.random_uniform([voc_size,embedding_size],-1.0,1.0))
    embed = tf.nn.embedding_lookup(embeddings, X) # lookup table
    
# Construct the variables for the NCE loss
nce_weights = tf.Variable(tf.random_uniform([voc_size, embedding_size],-1.0, 1.0))
nce_biases = tf.Variable(tf.zeros([voc_size]))

# Compute the average NCE loss for the batch.
# This does the magic:
#   tf.nn.nce_loss(weights, biases, inputs, labels, num_sampled, num_classes ...)
# It automatically draws negative samples when we evaluate the loss.
loss = tf.reduce_mean(tf.nn.nce_loss(nce_weights, nce_biases, Y, embed, num_sampled, voc_size))
# Use the adam optimizer
optimizer = tf.train.AdamOptimizer(1e-1).minimize(loss)

As we dicussed in the last story , word2vec model has a 3 layer neural network (input , hidden and output).

Here Hidden layer is just the dot product of inputs and weights ( no activation function here)

Output layer has an activation function ( here it is Noise-contrastive estimation ) we can use softmax also here but NCE is good you can read about it in that paper(NCE internally uses the softmax).

And we can define the size of hidden layer (here i took [voc_size, embedding_size(2)] but you can choose as your wish.

okay let’s train the model for 10000 iterations

In [0]:
epochs=10000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        
        batch_inputs, batch_labels = get_batch(batch_size)
        _,loss_val=sess.run([optimizer,loss],feed_dict = {X : batch_inputs, Y : batch_labels })
        
        if epoch % 1000 == 0:
            print("Loss at ", epoch, loss_val) # Report the loss
    
    # Final embeddings are ready for you to use. Need to normalize for practical use
    trained_embeddings = embeddings.eval()
    

import matplotlib.pyplot as plt
# Show word2vec if dim is 2
if trained_embeddings.shape[1] == 2:
    labels = unique_words[:500] # Show top 500 words
    for i, label in enumerate(labels):
        x, y = trained_embeddings[i,:]
        plt.scatter(x, y)
        plt.annotate(label, xy=(x, y), xytext=(5, 2),
            textcoords='offset points', ha='right', va='bottom')
    #plt.savefig("word2vec.png")
    plt.show()

So the error is decreased from 61 to 12 which is okay , now the model learned the weights so we got the final embeddings

Let’s visualize them for top 50 words

In [0]:
import matplotlib.pyplot as plt
# Show word2vec if dim is 2
if trained_embeddings.shape[1] == 2:
    labels = unique_words[:50] # Show top 50 words
    for i, label in enumerate(labels):
        x, y = trained_embeddings[i,:]
        plt.scatter(x, y)
        plt.annotate(label, xy=(x, y), xytext=(5, 2),
            textcoords='offset points', ha='right', va='bottom')
    #plt.savefig("word2vec.png")
    plt.show()